In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sktime.classification.deep_learning.mlp import MLPClassifier

from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from pyts.classification import *

from sktime.classification.sklearn import RotationForest
from sktime.classification.deep_learning.cnn import CNNClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sktime.classification.deep_learning.fcn import FCNClassifier

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from backtesting.test import EURUSD, SMA
import pmdarima as pm

data = EURUSD.copy()
data


# data = pd.read_table('/Users/newuser/Projects/robust-algo-trader/data/EURUSD_H1_200702210000_202304242100.tsv')
# # remove the following columns <TICKVOL>, <VOL> and <SPREAD>
# data = data.drop(['<TICKVOL>', '<VOL>', '<SPREAD>'], axis=1)
# # rename the columns
# data = data.rename(columns={'<DATE>': 'Date',
#                                 '<TIME>': 'Time',
#                                 '<OPEN>': 'Open',
#                                 '<HIGH>': 'High',
#                                 '<LOW>': 'Low',
#                                 '<CLOSE>': 'Close'})
# # combine the date and time columns
# data['Date_Time'] = data['Date'] + ' ' + data['Time']
# # convert the date_time column to datetime
# data['Date_Time'] = pd.to_datetime(data['Date_Time'], format='%Y%m%d %H:%M:%S.%f')
# # remove the date and time columns
# data = data.drop(['Date', 'Time'], axis=1)
# data.index = data['Date_Time']


def BBANDS(data, n_lookback, n_std):
    """Bollinger bands indicator"""
    hlc3 = (data.High + data.Low + data.Close) / 3
    mean, std = hlc3.rolling(n_lookback).mean(), hlc3.rolling(n_lookback).std()
    upper = mean + n_std*std
    lower = mean - n_std*std
    return upper, lower


close = data.Close.values
sma10 = SMA(data.Close, 10)
sma20 = SMA(data.Close, 20)
sma50 = SMA(data.Close, 50)
sma100 = SMA(data.Close, 100)
upper, lower = BBANDS(data, 20, 2)

# Design matrix / independent features:

# Price-derived features
data['X_SMA10'] = (close - sma10) / close
data['X_SMA20'] = (close - sma20) / close
data['X_SMA50'] = (close - sma50) / close
data['X_SMA100'] = (close - sma100) / close

data['X_DELTA_SMA10'] = (sma10 - sma20) / close
data['X_DELTA_SMA20'] = (sma20 - sma50) / close
data['X_DELTA_SMA50'] = (sma50 - sma100) / close

# Indicator features
data['X_MOM'] = data.Close.pct_change(periods=2)
data['X_BB_upper'] = (upper - close) / close
data['X_BB_lower'] = (lower - close) / close
data['X_BB_width'] = (upper - lower) / close
# data['X_Sentiment'] = ~data.index.to_series().between('2017-09-27', '2017-12-14')

# Some datetime features for good measure
data['X_day'] = data.index.dayofweek
data['X_hour'] = data.index.hour

# data = data.dropna()
data = data.dropna().astype(float)


def get_X(data):
    """Return model design matrix X"""
    return data.filter(like='X').values


def get_y(data):
    """Return dependent variable y"""
    y = data.Close.pct_change(48).shift(-48)  # Returns after roughly two days
    # Devalue returns smaller than 0.4%
    y[y.between(-.004, .004)] = 0
    y[y > 0] = 1
    y[y < 0] = -1
    return y


def get_clean_Xy(df):
    """Return (X, y) cleaned of NaN values"""
    X = get_X(df)
    y = get_y(df).values
    isnan = np.isnan(y)
    X = X[~isnan]
    y = y[~isnan]
    return X, y



X, y = get_clean_Xy(data)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=.5, random_state=0)



In [ ]:

class ClassificationBaseline:
    """Baseline model that always predicts the most frequent class"""
    def fit(self,X, y):
        train_labels = y
        train_label_counts = pd.Series(train_labels).value_counts()
        self.featureless_pred_label = train_label_counts.idxmax()

    def predict(self, X):
        test_features = X
        test_nrow, test_ncol = test_features.shape
        return np.repeat(self.featureless_pred_label, test_nrow)

    
tscv = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)

pred_score_list = []
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    # Model the output based on 7 "nearest" examples
    # clf = KNeighborsClassifier(7)
    # clf = LogisticRegressionCV(max_iter=100_000, cv=5, random_state=0)
    # clf = DecisionTreeClassifier( random_state=0)
    # clf = MLPClassifier()
    # clf = RidgeClassifierCV()
    # clf = RandomForestClassifier()
    # clf = RadiusNeighborsClassifier(radius=100.0)
    # clf = GradientBoostingClassifier()
    
    clf = TimeSeriesForest()
    # clf = LearningShapelets()
    
    # clf = RotationForest()
    # clf = TimeSeriesForestClassifier()
    # clf =  CNNClassifier(n_epochs=20,batch_size=4) 
    # clf =  FCNClassifier(n_epochs=20,batch_size=4) 
    # clf =  MLPClassifier(n_epochs=20,batch_size=4) 
    
    
    clf = ClassificationBaseline()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # # Fit your model
    # model = pm.auto_arima(train, seasonal=True, m=12)

    # # make your forecasts
    # forecasts = model.predict(test.shape[0])  # predict N steps into the future
    
    
    pred_score_list.append(
        pd.DataFrame({
            'accuracy': accuracy_score(y_test, y_pred)*100,
            'split': i+1,
            'Algorithm': 'KNN',
        }, index=[0]))

pred_score_df = pd.concat(pred_score_list)

print(pred_score_df['accuracy'])

# Plot a bar chart of accuracy vs split for each algorithm
plt.bar(pred_score_df["split"], pred_score_df["accuracy"], label=pred_score_df["Algorithm"][0])
plt.xlabel("split")
plt.ylabel("accuracy")
plt.title("Accuracy vs split for KNN algorithm")
plt.show()

# print('Classification accuracy: ', np.mean(y_test == y_pred))

In [ ]:
0    47.153465
0    46.658416
0    43.440594
0    51.361386
0    46.410891

In [ ]:
data = data.dropna()

In [ ]:
data
# use Yeo-Johnson transform from sklearn
from sklearn.preprocessing import PowerTransformer

ohlc_columns = data.columns

pt = PowerTransformer(method='yeo-johnson', standardize=True)
pt.fit(data)

# Transform the OHLC data
ohlc_data_transformed = pt.transform(data)

# Convert the transformed data to a dataframe
ohlc_df_transformed = pd.DataFrame(ohlc_data_transformed, columns=ohlc_columns)


In [ ]:
ohlc_df_transformed

In [ ]:
%%time

from backtesting import Backtest, Strategy
from pyts.classification import KNeighborsClassifier


N_TRAIN = 400

class MLTrainOnceStrategy(Strategy):
    price_delta = .004  # 0.4%

    def init(self):        
        # Init our model, a kNN classifier
        self.clf = KNeighborsClassifier()

        # Train the classifier in advance on the first N_TRAIN examples
        df = self.data.df.iloc[:N_TRAIN]
        X, y = get_clean_Xy(df)
        self.clf.fit(X, y)

        # Plot y for inspection
        self.I(get_y, self.data.df, name='y_true')

        # Prepare empty, all-NaN forecast indicator
        self.forecasts = self.I(lambda: np.repeat(np.nan, len(self.data)), name='forecast')
        
    
    def next(self):
        # Skip the cold start period with too few values available
        if len(self.data) < N_TRAIN:
            return
        
        if (len(self.data)) % N_TRAIN == 0:
            # Train the classifier in advance on the first N_TRAIN examples
            # get all rows except the current one
            N_TRAIN_UPDATE = len(self.data) - 1
            
            df = self.data.df.iloc[:N_TRAIN_UPDATE]
            X, y = get_clean_Xy(df)
            self.clf.fit(X, y)
        
        # Forecast the next movement
        X = get_X(self.data.df.iloc[-1:])
        forecast = self.clf.predict(X)[0]

        # Update the plotted "forecast" indicator
        self.forecasts[-1] = forecast

        # If our forecast is upwards and we don't already hold a long position
        # place a long order for 20% of available account equity. Vice versa for short.
        # Also set target take-profit and stop-loss prices to be one price_delta
        # away from the current closing price.
        # Proceed only with out-of-sample data. Prepare some variables
        
        high, low, close = self.data.High, self.data.Low, self.data.Close
        current_time = self.data.index[-1]
        
        upper, lower = close[-1] * (1 + np.r_[1, -1]*self.price_delta)

        if forecast == 1 and not self.position.is_long:
            self.buy(size=.2, tp=upper, sl=lower)
        elif forecast == -1 and not self.position.is_short:
            self.sell(size=.2, tp=lower, sl=upper)

        for trade in self.trades:
            if current_time - trade.entry_time > pd.Timedelta('2 days'):
                if trade.is_long:
                    trade.sl = max(trade.sl, low)
                else:
                    trade.sl = min(trade.sl, high)



bt = Backtest(data, MLTrainOnceStrategy, commission=.0002, margin=.05)
bt.run()
bt.plot()


In [ ]:
X, y = get_clean_Xy(ohlc_df_transformed)

In [ ]:
y